<a href="https://colab.research.google.com/github/LucaswasTaken/Acessibilidade_Parte_1_Analise_de_Sentimentos/blob/main/match_process/Automatic_Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Caro usuário, esse notebook é destinado a popular a fila de matchs, bem como realizar os matchs automáticos se o usuário assim desejar. Para utiliza-lo, é necessário antes gerar uma fila de segementos para match com o notebook que a gera com a formatação adequada e treinar um modelo do Annoy e ter ele salvo no storage com seu respectivo tokenizador. Todos esses arquivos podem ser encontrados na pasta "modelos" do storage, no diretório raiz do comunity.

# Instalação do Pytools e Variaveis do Firebase

In [1]:
from google.colab import auth
import sys
import subprocess

# Autenticação é necessária para clonar
auth.authenticate_user()


In [2]:
%%capture
!gcloud source repos clone github_hand-talk_pytools --project='ht-community' && pip install -r /content/github_hand-talk_pytools/requirements.txt
sys.path.insert(0, '/content/github_hand-talk_pytools')

In [3]:
from handtalk.community import Community

#@title Definição de variáveis globais  { form-width: "30%" }
collection = "segments" #@param ["animations", "videos", "contribs", "segments", "signs", "votes"] {allow-input: true}
WORKSPACE = "HT-BZS" #@param ["HT-Annoy", "HT-BZS", "development", "HT-BZS-JOAB", "HT-ASL"] {allow-input: true}
cmt = Community(WORKSPACE)

🍒 Using Cereja v.1.3.7
{'Faça o upload do arquivo json do service accounts do community. Se não tiver um, peça ao Thadeu.'}


Saving ht-community-firebase-adminsdk-pazxx-a4c2b612bb.json to ht-community-firebase-adminsdk-pazxx-a4c2b612bb.json


In [4]:
# Carregando dados do storage
if WORKSPACE == 'HT-ASL':
    cmt.storage.download(f'modelos/joab_model_class/asl/class_model_{WORKSPACE}.hdf5','./')
    cmt.storage.download(f'modelos/joab_model_class/asl/class_model_{WORKSPACE}.txt','./')
    cmt.storage.download(f'modelos/joab_model_class/asl/tokenizer_{WORKSPACE}.json','./')
    cmt.storage.download(f'modelos/annoy/asl/{WORKSPACE}.json','./')
    cmt.storage.download(f'modelos/annoy/asl/{WORKSPACE}.ann','./')
    cmt.storage.download(f'modelos/siamese/siamese_keypoints1.h5','./')
    cmt.storage.download(f'modelos/automatic_match/matched_{WORKSPACE}.txt','./')
    cmt.storage.download(f'modelos/automatic_match/old_videos_{WORKSPACE}.txt','./')
    cmt.storage.download(f'modelos/automatic_match/priority_segments_{WORKSPACE}.json','./')

else:
    cmt.storage.download(f'modelos/joab_model_class/bzs/class_model_{WORKSPACE}.hdf5','./')
    cmt.storage.download(f'modelos/joab_model_class/bzs/tokenizer_{WORKSPACE}.json','./')
    cmt.storage.download(f'modelos/annoy/bzs/{WORKSPACE}.json','./')
    cmt.storage.download(f'modelos/annoy/bzs/{WORKSPACE}.ann','./')
    cmt.storage.download(f'modelos/siamese/siamese_keypoints1.h5','./')
    cmt.storage.download(f'modelos/automatic_match/matched_{WORKSPACE}.txt','./')
    cmt.storage.download(f'modelos/automatic_match/priority_segments_{WORKSPACE}.json','./')

# Funções gerais e importaçã dos modelos

In [5]:
def rescale_list_ht(segment, granularity):
        cluster = int(len(segment) / granularity)
        if (cluster == 0):
            multiplicador = int(granularity / len(segment) + 1)
            oversampling = []

            for array in segment:
                for i in range(multiplicador):
                    oversampling.append(array)

            segment = oversampling
            cluster = 1

        flatten_result = []
        start_interval = 0

        for i in range(granularity):
            frames = segment[start_interval:start_interval + cluster]
            flatten_result.append(frames[-1])
            start_interval += cluster
        
        assert len(flatten_result) == granularity, f"Erro ao reescalar tamanho da lista {len(flatten_result)} != {granularity}"
        return flatten_result

### Rede siamesa do Lucas

In [6]:
from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
import keras
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.models import Model, load_model
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D
from keras import backend as K

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

siamese_model = load_model('/content/siamese_keypoints1.h5')

In [7]:
LANGUAGE = 'eng' if WORKSPACE != 'HT-BZS' else 'por'
LANGUAGE_ID = 'ase' if LANGUAGE == 'eng' else 'bzs'
print(f'Trabalhando em "{WORKSPACE}"')

Trabalhando em "HT-BZS"


### ANNOY

In [8]:
!pip install annoy

     |████████████████████████████████| 655kB 5.1MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391566 sha256=91d665d9ed8e8b63936acd727cb3044c1ea906487096b03d993dbe08376ae115
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [9]:
from annoy import AnnoyIndex
import random

f = 840
u = AnnoyIndex(f, 'manhattan')
u.load(f'{WORKSPACE}.ann') # super fast, will just mmap the file

True

### Funções do Modelo do Joab

In [10]:
import json
with open(f'{WORKSPACE}.json') as fi:
    tokenizer = json.load(fi)

detokenizer = {}
for token in tokenizer:
    detokenizer[tokenizer[token]] = token


In [11]:
from handtalk.gcloud.firebase import Firebase, Firestore
from handtalk.community import Community
from copy import deepcopy
from cereja import Progress, TfIdf
import cereja as cj
from typing import List, Set, Tuple, Dict
p = Progress()

In [12]:


"""
Class for managing our data.
"""
import cereja as cj
import csv
import numpy as np
import random
import glob
import os.path
import sys
import operator
import threading
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import itertools

class threadsafe_iterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return next(self.iterator)

def threadsafe_generator(func):
    """Decorator"""
    def gen(*a, **kw):
        return threadsafe_iterator(func(*a, **kw))
    return gen


def process_image(image, target_shape=(299, 299, 3)):
    img = load_img(image, target_size=target_shape[:2])
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x[0]

class DataSet():

    def __init__(self, data, classes, seq_length=10, image_shape=(299, 299, 3)):
        """Constructor.
        seq_length = (int) the number of frames to consider
        class_limit = (int) number of classes to limit the data to.
            None = no limit.
        """
        self.seq_length = seq_length
        # self.model_features = Extractor('/content/drive/MyDrive/_video_frames_community/inception_best.hdf5')
        
        train_total = int(len(data) * 0.8)
        random.shuffle(data)
        self.test_data = data[train_total:]
        
        
        self.train_data = data[:train_total]
        self.data_length = len(self.train_data)
        self.classes = sorted(classes)

        self.image_shape = image_shape
    

    def get_class_one_hot(self, class_str):
        """Given a class as a string, return its number in the classes
        list. This lets us encode and one-hot it for training."""
        # Encode it first.
        label_encoded = self.classes.index(class_str)

        # Now one-hot it.
        label_hot = to_categorical(label_encoded, len(self.classes))

        assert len(label_hot) == len(self.classes)

        return label_hot
        
    @threadsafe_generator
    def frame_generator_binary(self, batch_size, is_train=True, is_features=False):

        while 1:
            X1, X2, y = [], [], []
            

            # Generate batch_size samples.
            for _ in range(batch_size):
                # Reset to be safe.
                a, b, c = None, None, None
                
                if is_train:
                    a, b, c = random.choice(self.train_data)
                else:
                    a, b, c = random.choice(self.test_data)

                if is_features:
                    a = self.get_features(a)
                    b = self.get_features(b)
                else:
                    # Get and resample frames.
                    a = self.get_frames_for_sample(a)
                    b = self.get_frames_for_sample(b)

                    a = self.rescale_list_ht(a, self.seq_length)
                    b = self.rescale_list_ht(b, self.seq_length)

                    # Build the image sequence
                    a = self.build_image_sequence(a)
                    b = self.build_image_sequence(b)

                X1.append(a)
                X2.append(b)
                y.append(c)
            yield [np.array(X1), np.array(X2)], np.array(y)


    @threadsafe_generator
    def frame_generator_categorical(self, batch_size, is_train=True, is_features=False):

        while 1:
            X, y = [], []
            

            # Generate batch_size samples.
            for _ in range(batch_size):
                # Reset to be safe.
                a, b = None, None
                
                if is_train:
                    a, b = random.choice(self.train_data)
                else:
                    a, b = random.choice(self.test_data)

                if is_features:
                    a = self.get_features(a)
                else:
                    # Get and resample frames.
                    a = self.get_frames_for_sample(a)

                    a = self.rescale_list_ht(a, self.seq_length) # 
                    
                    # Build the image sequence
                    a = self.build_image_sequence(a)

                X.append(a)
                y.append(self.get_class_one_hot(b))
            yield np.array(X), np.array(y)

    def get_features(self, feature_path):
        return np.load(feature_path)
    
    @classmethod
    def rescale_list_ht(cls, segment, granularity):
        if len(segment) >= granularity:
            return cls.rescale_list(segment, granularity)
        if (len(segment) == 0):
            return []
        cluster = int(len(segment) / granularity)
        if (cluster == 0):
            multiplicador = int(granularity / len(segment) + 1)
            oversampling = []

            for array in segment:
                for i in range(multiplicador):
                    oversampling.append(array)

            segment = oversampling
            cluster = 1

        flatten_result = []
        start_interval = 0

        for i in range(granularity):
            frames = segment[start_interval:start_interval + cluster]
            flatten_result.append(frames[-1])
            start_interval += cluster
        
        assert len(flatten_result) == granularity, f"Erro ao reescalar tamanho da lista {len(flatten_result)} != {granularity}"
        return flatten_result


    def build_image_sequence(self, frames):
        """Given a set of frames (filenames), build our sequence."""
        return [process_image(x, self.image_shape) for x in frames]


    def get_frames_for_sample(self, sample):
        """Given a sample row from the data file, get all the corresponding frame
        filenames."""
        return list(sorted(sample, key=lambda x: int(x.rsplit('_', 1)[-1].split('.')[0])))


    @staticmethod
    def rescale_list(input_list, size):
        """Given a list and a size, return a rescaled/samples list. For example,
        if we want a list of size 5 and we have a list of size 25, return a new
        list of size five which is every 5th element of the origina list."""
        assert len(input_list) >= size, f'{len(input_list), size}'

        # Get the number to skip between iterations.
        skip = len(input_list) // size

        # Build our new output.
        output = [input_list[i] for i in range(0, len(input_list), skip)]

        # Cut off the last one if needed.
        return output[:size]


    def class_from_prediction(self, predictions, nb_to_return=5):
        """Given a prediction, print the top classes."""
        # Get the prediction for each label.
        label_predictions = {}
        for i, label in enumerate(self.classes):
            label_predictions[label] = predictions[i]

        # Now sort them.
        sorted_lps = sorted(
            label_predictions.items(),
            key=operator.itemgetter(1),
            reverse=True
        )
        result = []
        # And return the top N.
        for i, class_prediction in enumerate(sorted_lps):
            if i > nb_to_return - 1 or class_prediction[1] == 0.0:
                break
            result.append((class_prediction[0], class_prediction[1]))
        return result

## Resgatar lista de dados do firebase

In [13]:
# atualize os snapshots dos segmentos aqui
where = [('keypointsExtracted', '==', True)]
videos_extracted = cmt.list_longer_collection('videos', where_config=where)
len(videos_extracted)

🍒 Sys[out] » Total videos 30000 query time 26.74s 
🍒 Sys[out] » Total videos 59677 query time 53.46s 
🍒 Cereja » [▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰] - 100.00% - 🕜 00:00:53 total - Done! ✅ 


59677

In [14]:
video_extracted_set = set(videos_extracted.keys())

In [15]:
if WORKSPACE == 'HT-ASL':
    videos_old = set(cj.FileIO.load(f'/content/old_videos_{WORKSPACE}.txt').data)
    segments_old = set(cj.FileIO.load(f'/content/matched_{WORKSPACE}.txt').data)
    list_segments_old = list(cj.FileIO.load(f'/content/matched_{WORKSPACE}.txt').data)
else:
    segments_old = set(cj.FileIO.load(f'/content/matched_{WORKSPACE}.txt').data)
    list_segments_old = list(cj.FileIO.load(f'/content/matched_{WORKSPACE}.txt').data)

In [16]:
segments_to_match = cj.FileIO.load(f'/content/priority_segments_{WORKSPACE}.json').data

# Definindo segmentos similares

### Antigo modelo do Joab

In [17]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
import numpy as np
import cereja as cj
import json

# Carregando modelos

if WORKSPACE == 'HT-ASL':
    joab_model = load_model("/content/class_model_HT-ASL.hdf5")

    classes = cj.FileIO.load('/content/class_model_HT-ASL.txt').data
    dataset = DataSet(
        data=[],
        classes=classes,
        seq_length=10,
        image_shape=(299, 299, 3)
    )

    with open("tokenizer_HT-ASL.json") as output:
        p_segments = json.load(output)

else:
    with open('/content/tokenizer_HT-BZS.json') as f:
        joab_sign_dict = json.load(f)
    joab_model = load_model(f"/content/class_model_{WORKSPACE}.hdf5")

    with open(f"tokenizer_{WORKSPACE}.json") as output:
        p_segments = json.load(output)

In [18]:
import keras
import json
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import glob

!mkdir temp_data

def process_image(image, target_shape):
    """Given an image, process it and return the array."""
    # Load the image.
    h, w, _ = target_shape
    image = load_img(image, target_size=(h, w))

    # Turn it into numpy, normalize and return.
    img_arr = img_to_array(image)
    x = np.asarray(img_arr / 255.).astype('float32')
    return x

# Modelo original do Joab pra indicar a que cluster o segmento pertence
def predict_segment_cluster(segment_id,workspace):
    if workspace == 'HT-ASL':
        !rm -r temp_data/*
        candidates = []
        document = cmt.get_document('segments', segment_id)
        document.save_frames('temp_data', (299, 299))
        images = dataset.get_frames_for_sample(map(lambda x: x, glob.glob('/content/temp_data/*'))) # ordena frames
        images = dataset.rescale_list_ht(images, 10)
        images = dataset.build_image_sequence(images)
        x = np.array([images])
        prediction = joab_model.predict(x)
        result = dataset.class_from_prediction(np.squeeze(prediction, axis=0), 1)
        for sign_id, rate in result:
            if rate > 0.8:
                candidates.append([p_segments[sign_id],-0.99])
                return candidates
            elif rate > 0.6:
                candidates.append([p_segments[sign_id],-0.90])
                return candidates
            else:
                return candidates
    else:
        !rm -r ./temp_data/*
        image_shape = (150, 150, 3)
        cmt.get_document('segments', segment_id).save_frames('./temp_data/')
        frames_path = [i for i in glob.glob('/content/temp_data/*')]
        frames_path = list(sorted(frames_path, key=lambda x: int(x.rsplit('_')[-1].split('.')[0])))

        frames = rescale_list_ht(frames_path, 10)
        sample = np.array([[process_image(i, image_shape) for i in frames]])
        
        # Predict!
        prediction = joab_model.predict(sample)
        
        candidates = [[joab_sign_dict[str(np.array(prediction[0]).argsort()[-1:][0])],-0.8]]
        return candidates

In [19]:
# Lista de segmentos presentes no modelo do Annoy
list_annoy = set(detokenizer.keys())

# Verifica membro commum em 2 frases
def common_member(a, b): 
    a_set = set(a) 
    b_set = set(b) 
    if (a_set & b_set): 
        return True 
    else: 
        return False

In [20]:
# Template de voto automático
from datetime import datetime

vote_template = {
    'createdAt' : datetime.now(),
    'lastUpdate': datetime.now(),
    'score': 2,
    'votes': 1,
    'automaticMatch': True,
    'segment_a': '',
    'segment_b': ''
}

In [21]:
# Faz upload das atualizações para o storage
def upload_automatic_segments(filepath: str, workspace: str):
    blob = Community.firebase.storage.db.blob(f'modelos/automatic_match/matched_{workspace}.txt')
    blob.upload_from_filename(filepath)

    blob = Community.firebase.storage.db.blob(f'modelos/automatic_match/priority_segments_{workspace}.json')
    blob.upload_from_filename(f'/content/priority_segments_{workspace}.json')

In [22]:
# Transforma em conjunto a lista de Match pra não haver repetição
segments_list = set(segments_to_match.keys())

## Formação da fila de match e matchs automáticos

In [ ]:
from handtalk import utils
from copy import deepcopy
from tqdm import tqdm
from IPython.display import clear_output

vote_col = cmt.firestore.db.collection('workspaces').document(WORKSPACE).collection('votes')
false_match = {}
count = 0
segments_old = list(list_segments_old)
want_auto_match= input('Matchar automaticamente? (sim ou nao) ')
confidence= int(input('qual a confiança? (10-100) '))
for seg_id in tqdm(segments_list):
    # Segmento fora do annoy
    if seg_id not in list_annoy:
      continue
    # Segmento ja matchado
    if seg_id in segments_old:
      continue

    # Utilizando a rede do Joab com muita confiabilidade
    try:
        false_cluster = predict_segment_cluster(seg_id,WORKSPACE)
    except:
        false_cluster = []
    # Utilziando rede siamesa do Lucas
    candidates = {}
    candidates['segment_similar'] = []

    # Resgatar 50 segmentos similares do annoy
    for segment_similar in u.get_nns_by_item(int(detokenizer[seg_id]), confidence+1):
        if tokenizer[str(segment_similar)] == seg_id:
            continue
        candidates['segment_similar'].append(tokenizer[str(segment_similar)])
    candidates['keypoints_similar'] = []

    segment_data = cmt.get_document('segments',seg_id)
    
    # Resgatando Keypoints
    keypoints = segment_data.video.keypoints_array[segment_data.start_frame:segment_data.end_frame]
    actual_keypoint = [np.reshape(np.array(rescale_list_ht(keypoints, 10)),(10,177))]

    for candidate in candidates['segment_similar']:
        segment_data = cmt.get_document('segments',candidate)
        keypoints = segment_data.video.keypoints_array[segment_data.start_frame:segment_data.end_frame]
        candidates['keypoints_similar'].append(np.reshape(np.array(rescale_list_ht(keypoints, 10)),(10,177)))

    results = []
    actual_keypoint_list = []
    for i in range(confidence):
        actual_keypoint_list.append(actual_keypoint[0])
    # Aplicando rede siamesa
    results = -siamese_model.predict([np.array(actual_keypoint_list[0:confidence]),np.array(candidates['keypoints_similar'][0:confidence])])
    results = np.reshape(results,(len(results)))

    candidates['similares'] = false_cluster

    # Aplicando rede siamesa
    for seg_id_result in np.array(results).argsort()[0:10]:
        candidates['similares'].append([candidates['segment_similar'][seg_id_result],results[seg_id_result]])
        if len(candidates['similares']) == 10:
            break
    
    false_match[seg_id] = candidates['similares']

    list_segments_old.append(seg_id)
    count+=1
    clear_output(wait = True)
    if count%50 == 0:
        real_pairs = []
        up_operations = []
        
        for seg_key in false_match:
            if len(false_match[seg_key]) > 0:
                main_sentence = utils.preprocess(cmt.get_document('segments',seg_key).video.sentence).split()
                # aux_queue = True
                if want_auto_match == 'sim':
                    for sec_seg,score in false_match[seg_key][0:3]:
                        sec_sentence = utils.preprocess(cmt.get_document('segments',sec_seg).video.sentence).split()
                        if (common_member(main_sentence,sec_sentence)) and (-score > 0.985):
                            real_pairs.append(sorted([seg_key,sec_seg]))
                            aux_queue = False
                # if aux_queue:
                segments_to_match[seg_key]['similarSegments'] = [row[0] for row in false_match[seg_key]]

        for linked in real_pairs:
            to_link = sorted(linked)
            id_doc = cmt.firestore.db.collection('whatever').document().id
            vote_ref = vote_col.document(id_doc)
            vote_data = deepcopy(vote_template)
            vote_data['segment_a'] = to_link[0]
            vote_data['segment_b'] = to_link[1]
            vote_data['createdAt'] = datetime.now()
            vote_data['lastUpdate'] = datetime.now()
            up_operations.append((vote_ref, vote_data))


        f=open(f'/content/matched_{WORKSPACE}.txt','w')
        for ele in list_segments_old:
            f.write(ele+'\n')
        f.close()
        
        with open(f'/content/priority_segments_{WORKSPACE}.json','w') as f:
            json.dump(segments_to_match,f)

        upload_automatic_segments(f'/content/matched_{WORKSPACE}.txt', WORKSPACE)

        if len(up_operations) > 0:
            cmt.firestore.set(up_operations,safe_mode=False)
            # print(up_operations)
            # cmt.firestore.set(up_operations)
        false_match = {}

  3%|▎         | 638/19867 [2:08:19<88:34:50, 16.58s/it] 